<a href="https://colab.research.google.com/github/56sarager/Colabs/blob/main/Yahoo_Finance_Dashboard.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Financials Dashboard

In [ ]:
import yfinance as yf
import pandas as pd
import numpy as np
from plotly.subplots import make_subplots
import plotly.graph_objects as go
from IPython.display import display

def calculate_ratios(financials, balance_sheet, cashflow, info, history):
    ratios = {}
    try:
        ratios["EPS"] = (financials["Net Income"] *(history(period="1d")['Close'].iloc[-1]/ info["marketCap"])).dropna()
        ratios["P/E"] = history(period="1d")['Close'].iloc[-1]/ ratios["EPS"]
        # Profitability Ratios
        ratios["Gross Margin"] = (financials["Gross Profit"] / financials["Total Revenue"]).dropna()*100
        ratios["Profit Margin"] = (financials["Net Income"] / financials["Total Revenue"]).dropna()*100
        ratios["Operating Margin"] = (financials["Operating Income"] / financials["Total Revenue"]).dropna()*100
        ratios["Return on Assets"] = (financials["Net Income"] / balance_sheet["Total Assets"]).dropna()
        ratios["Return on Equity"] = (financials["Net Income"] / balance_sheet["Stockholders Equity"]).dropna()
        ratios["Return on Total Capital"] = (financials["Pretax Income"] / (balance_sheet["Stockholders Equity"] + balance_sheet["Total Debt"])).dropna()

        # Liquidity Ratios
        ratios["Leverage"] = (balance_sheet["Total Assets"] / balance_sheet["Stockholders Equity"]).dropna()
        ratios["Current Ratio"] = (balance_sheet["Current Assets"] / balance_sheet["Current Liabilities"]).dropna()
        ratios["Quick Ratio"] = ((balance_sheet["Current Assets"] - balance_sheet["Inventory"]) / balance_sheet["Current Liabilities"]).dropna()
        ratios["Cash Ratio"] = (balance_sheet["Cash And Cash Equivalents"] / balance_sheet["Current Liabilities"]).dropna()

        # Leverage Ratios
        ratios["Debt to EBITDA"] = (balance_sheet["Total Liabilities Net Minority Interest"] / financials["EBITDA"]).dropna()
        ratios["Debt to Asset"] = (balance_sheet["Total Liabilities Net Minority Interest"] / balance_sheet["Total Assets"]).dropna()
        ratios["Debt to Equity"] = (balance_sheet["Total Liabilities Net Minority Interest"] / balance_sheet["Stockholders Equity"]).dropna()
        ratios["Debt to Capital"] = (balance_sheet["Total Liabilities Net Minority Interest"] / (balance_sheet["Total Liabilities Net Minority Interest"] + balance_sheet["Stockholders Equity"])).dropna()

        # Efficiency Ratios
        ratios["Assets Turnover"] = (financials["Total Revenue"] / balance_sheet["Total Assets"]).dropna()
        ratios["Inventory Turnover"] = (financials["Cost Of Revenue"] / balance_sheet["Inventory"]).dropna()
        ratios["Debtor Turnover"] = (financials["Total Revenue"] / balance_sheet["Accounts Receivable"]).dropna()
        ratios["Accounts Payable Turnover"] = (financials["Cost Of Revenue"] / balance_sheet["Accounts Payable"]).dropna()
    except KeyError as e:
        print(f"Key missing in data: {e}")
    return ratios

ticker_symbol = "AAPL"
interval = 'quarters'
data_range = input("Enter 'quarters' for past 5 quarters or 'years' for past 5 years (default quarters): ").strip().lower() or interval
ticker = input("Enter ticker symbol (default AAPL): ").strip().upper() or ticker_symbol
ticker_2 = input("Optional second ticker for comparison: ").strip().upper()

stock = yf.Ticker(ticker)
if data_range == "quarters":
    financials = stock.quarterly_financials.T
    balance_sheet = stock.quarterly_balance_sheet.T
    cashflow = stock.quarterly_cashflow.T
    info = stock.info
    history = stock.history
else:
    financials = stock.financials.T
    balance_sheet = stock.balance_sheet.T
    cashflow = stock.cashflow.T
    info = stock.info
    history = stock.history
if ticker_2:
    stock_2 = yf.Ticker(ticker_2)
    financials_2 = stock_2.quarterly_financials.T if data_range == "quarters" else stock_2.financials.T
    balance_sheet_2 = stock_2.quarterly_balance_sheet.T if data_range == "quarters" else stock_2.balance_sheet.T
    cashflow_2 = stock_2.quarterly_cashflow.T if data_range == "quarters" else stock_2.cashflow.T
    revenue2 = financials_2["Total Revenue"].dropna()
    free_cash_flow2 = cashflow_2["Free Cash Flow"].dropna()
    ebitda2 = financials_2["EBITDA"].dropna()
    info2 = stock.info
    history2 = stock.history


revenue = financials["Total Revenue"].dropna()
free_cash_flow = cashflow["Free Cash Flow"].dropna()
ebitda = financials["EBITDA"].dropna()

ratios = calculate_ratios(financials, balance_sheet, cashflow, info, history)
ratios_2 = calculate_ratios(financials_2, balance_sheet_2, cashflow_2, info2, history2) if ticker_2 else None

fig = make_subplots(
    rows=4, cols=3,
    specs=[[{"colspan": 2}, None, {"type": "scatter"}],
           [{"colspan": 2}, None, {"type": "scatter"}],
           [{"type": "scatter"}, {"type": "scatter"}, {"type": "scatter"}],
           [{"type": "scatter"}, {"colspan": 2}, None]],
    subplot_titles=("Revenue", "Earnings Per Share", "Capital",
                    "Profitability Ratios", "Liquidity Ratios", "Leverage Ratios", "Efficiency Ratios", "P/E", "EBITDA (Bullshit Earnings)"))

fig.add_trace(go.Bar(x=revenue.index, y=revenue.values, name=f"Revenue {ticker}", marker = {'color' : 'dimgrey'} ), row=1, col=1)
if ticker_2:
    fig.add_trace(go.Bar(x=revenue2.index, y=revenue2.values, name=f"Revenue {ticker_2}", marker = {'color' : 'darkgrey'} ), row=1, col=1)

fig.add_trace(go.Scatter(x=ratios["EPS"].index, y=ratios["EPS"].values, name=f"EPS {ticker}", marker = {'color' : 'paleturquoise'}), row=1, col=3)
if ticker_2:
    fig.add_trace(go.Scatter(x=ratios_2["EPS"].index, y=ratios_2["EPS"].values, name=f"EPS {ticker_2}", marker = {'color' : 'mediumturquoise'}), row=1, col=3)

fig.add_trace(go.Bar(x=free_cash_flow.index, y=free_cash_flow.values, name=f"Free Cash Flow {ticker}", marker = {'color' : 'silver'}), row=2, col=1)
if ticker_2:
    fig.add_trace(go.Bar(x=free_cash_flow2.index, y=free_cash_flow2.values, name=f"Free Cash Flow {ticker_2}", marker = {'color' : 'gainsboro'}), row=2, col=1)
working_capital = (balance_sheet["Total Assets"] - balance_sheet["Total Liabilities Net Minority Interest"]).dropna()
operating_capital = (balance_sheet["Current Assets"] - balance_sheet["Current Liabilities"] - balance_sheet["Cash And Cash Equivalents"]).dropna()
if ticker_2:
    working_capital_2 = (balance_sheet_2["Total Assets"] - balance_sheet_2["Total Liabilities Net Minority Interest"]).dropna()
    operating_capital_2 = (balance_sheet_2["Current Assets"] - balance_sheet_2["Current Liabilities"] - balance_sheet["Cash And Cash Equivalents"]).dropna()

fig.add_trace(go.Bar(x=working_capital.index, y=working_capital.values, name=f"Working Capital {ticker}", marker={'color': 'honeydew'}),row=2, col=1)
fig.add_trace(go.Bar(x=operating_capital.index, y=operating_capital.values, name=f"Operating Capital {ticker}", marker={'color': 'snow'}),row=2, col=1)
if ticker_2:
    fig.add_trace(go.Bar(x=working_capital_2.index, y=working_capital_2.values, name=f"Working Capital {ticker_2}", marker={'color': 'oldlace'}),row=2, col=1)
    fig.add_trace(go.Bar(x=operating_capital_2.index, y=operating_capital_2.values, name=f"Operating Capital {ticker_2}", marker={'color': 'lavenderblush'}),row=2, col=1)

fig.add_trace(go.Scatter(x=ratios["Gross Margin"].index, y=ratios["Gross Margin"].values, name=f"Gross Margin {ticker}", marker = {'color' : 'blue'}), row=2, col=3)
fig.add_trace(go.Scatter(x=ratios["Profit Margin"].index, y=ratios["Profit Margin"].values, name=f"Profit Margin {ticker}", marker = {'color' : 'cornflowerblue'}), row=2, col=3)
fig.add_trace(go.Scatter(x=ratios["Operating Margin"].index, y=ratios["Operating Margin"].values, name=f"Operating Margin {ticker}", marker = {'color' : 'lightsteelblue'}), row=2, col=3)
fig.add_trace(go.Scatter(x=ratios["Return on Assets"].index, y=ratios["Return on Assets"].values, name=f"Return on Assets {ticker}", marker = {'color' : 'steelblue'}), row=2, col=3)
fig.add_trace(go.Scatter(x=ratios["Return on Equity"].index, y=ratios["Return on Equity"].values, name=f"Return on Equity {ticker}", marker = {'color' : 'darkturquoise'}), row=2, col=3)
fig.add_trace(go.Scatter(x=ratios["Return on Total Capital"].index, y=ratios["Return on Total Capital"].values, name=f"Return on Total Capital {ticker}", marker = {'color' : 'powderblue'}), row=2, col=3)
if ticker_2:
    fig.add_trace(go.Scatter(x=ratios_2["Gross Margin"].index, y=ratios_2["Gross Margin"].values, name=f"Gross Margin {ticker_2}", marker = {'color' : 'teal'}), row=2, col=3)
    fig.add_trace(go.Scatter(x=ratios_2["Profit Margin"].index, y=ratios_2["Profit Margin"].values, name=f"Profit Margin {ticker_2}", marker = {'color' : 'cyan'}), row=2, col=3)
    fig.add_trace(go.Scatter(x=ratios_2["Operating Margin"].index, y=ratios_2["Operating Margin"].values, name=f"Operating Margin {ticker_2}", marker = {'color' : 'royalblue'}), row=2, col=3)
    fig.add_trace(go.Scatter(x=ratios_2["Return on Assets"].index, y=ratios_2["Return on Assets"].values, name=f"Return on Assets {ticker_2}", marker = {'color' : 'darkturquoise'}), row=2, col=3)
    fig.add_trace(go.Scatter(x=ratios_2["Return on Equity"].index, y=ratios_2["Return on Equity"].values, name=f"Return on Equity {ticker_2}", marker = {'color' : 'lightskyblue'}), row=2, col=3)
    fig.add_trace(go.Scatter(x=ratios_2["Return on Total Capital"].index, y=ratios_2["Return on Total Capital"].values, name=f"Return on Total Capital {ticker_2}", marker = {'color' : 'lightseagreen'}), row=2, col=3)

fig.add_trace(go.Scatter(x=ratios["Leverage"].index, y=ratios["Leverage"].values, name=f"Leverage {ticker}", marker = {'color' : 'blueviolet'}), row=3, col=1)
fig.add_trace(go.Scatter(x=ratios["Current Ratio"].index, y=ratios["Current Ratio"].values, name=f"Current Ratio {ticker}", marker = {'color' : 'darkviolet'}), row=3, col=1)
fig.add_trace(go.Scatter(x=ratios["Quick Ratio"].index, y=ratios["Quick Ratio"].values, name=f"Quick Ratio {ticker}", marker = {'color' : 'mediumorchid'}), row=3, col=1)
fig.add_trace(go.Scatter(x=ratios["Cash Ratio"].index, y=ratios["Cash Ratio"].values, name=f"Cash Ratio {ticker}", marker = {'color' : 'orchid'}), row=3, col=1)
if ticker_2:
    fig.add_trace(go.Scatter(x=ratios["Leverage"].index, y=ratios["Leverage"].values, name=f"Leverage {ticker_2}", marker = {'color' : 'rebeccapurple'}), row=3, col=1)
    fig.add_trace(go.Scatter(x=ratios_2["Current Ratio"].index, y=ratios_2["Current Ratio"].values, name=f"Current Ratio {ticker_2}", marker = {'color' : 'plum'}), row=3, col=1)
    fig.add_trace(go.Scatter(x=ratios_2["Quick Ratio"].index, y=ratios_2["Quick Ratio"].values, name=f"Quick Ratio {ticker_2}", marker = {'color' : 'violet'}), row=3, col=1)
    fig.add_trace(go.Scatter(x=ratios_2["Cash Ratio"].index, y=ratios_2["Cash Ratio"].values, name=f"Cash Ratio {ticker_2}", marker = {'color' : 'magenta'}), row=3, col=1)

fig.add_trace(go.Scatter(x=ratios["Debt to EBITDA"].index, y=ratios["Debt to EBITDA"].values, name=f"Debt to EBITDA {ticker}", marker = {'color' : 'lightcoral'}), row=3, col=2)
fig.add_trace(go.Scatter(x=ratios["Debt to Asset"].index, y=ratios["Debt to Asset"].values, name=f"Debt to Asset {ticker}", marker = {'color' : 'indianred'}), row=3, col=2)
fig.add_trace(go.Scatter(x=ratios["Debt to Equity"].index, y=ratios["Debt to Equity"].values, name=f"Debt to Equity {ticker}", marker = {'color' : 'firebrick'}), row=3, col=2)
fig.add_trace(go.Scatter(x=ratios["Debt to Capital"].index, y=ratios["Debt to Capital"].values, name=f"Debt to Capital {ticker}", marker = {'color' : 'salmon'}), row=3, col=2)
if ticker_2:
    fig.add_trace(go.Scatter(x=ratios_2["Debt to EBITDA"].index, y=ratios_2["Debt to EBITDA"].values, name=f"Debt to EBITDA {ticker_2}", marker = {'color' : 'coral'}), row=3, col=2)
    fig.add_trace(go.Scatter(x=ratios_2["Debt to Asset"].index, y=ratios_2["Debt to Asset"].values, name=f"Debt to Asset {ticker_2}", marker = {'color' : 'orangered'}), row=3, col=2)
    fig.add_trace(go.Scatter(x=ratios_2["Debt to Equity"].index, y=ratios_2["Debt to Equity"].values, name=f"Debt to Equity {ticker_2}", marker = {'color' : 'tomato'}), row=3, col=2)
    fig.add_trace(go.Scatter(x=ratios_2["Debt to Capital"].index, y=ratios_2["Debt to Capital"].values, name=f"Debt to Capital {ticker_2}", marker = {'color' : 'mistyrose'}), row=3, col=2)

fig.add_trace(go.Scatter(x=ratios["Assets Turnover"].index, y=ratios["Assets Turnover"].values, name=f"Assets Turnover {ticker}", marker = {'color' : 'darkorange'}), row=3, col=3)
fig.add_trace(go.Scatter(x=ratios["Inventory Turnover"].index, y=ratios["Inventory Turnover"].values, name=f"Inventory Turnover {ticker}", marker = {'color' : 'orange'}), row=3, col=3)
fig.add_trace(go.Scatter(x=ratios["Debtor Turnover"].index, y=ratios["Debtor Turnover"].values, name=f"Debtor Turnover {ticker}", marker = {'color' : 'darkgoldenrod'}), row=3, col=3)
fig.add_trace(go.Scatter(x=ratios["Accounts Payable Turnover"].index, y=ratios["Accounts Payable Turnover"].values, name=f"Accounts Payable Turnover {ticker}", marker = {'color' : 'palegoldenrod'}), row=3, col=3)
if ticker_2:
    fig.add_trace(go.Scatter(x=ratios_2["Assets Turnover"].index, y=ratios_2["Assets Turnover"].values, name=f"Assets Turnover {ticker_2}", marker = {'color' : 'goldenrod'}), row=3, col=3)
    fig.add_trace(go.Scatter(x=ratios_2["Inventory Turnover"].index, y=ratios_2["Inventory Turnover"].values, name=f"Inventory Turnover {ticker_2}", marker = {'color' : 'navajowhite'}), row=3, col=3)
    fig.add_trace(go.Scatter(x=ratios_2["Debtor Turnover"].index, y=ratios_2["Debtor Turnover"].values, name=f"Debtor Turnover {ticker_2}", marker = {'color' : 'gold'}), row=3, col=3)
    fig.add_trace(go.Scatter(x=ratios_2["Accounts Payable Turnover"].index, y=ratios_2["Accounts Payable Turnover"].values, name=f"Accounts Payable Turnover {ticker_2}", marker = {'color' : 'lemonchiffon'}), row=3, col=3)

fig.add_trace(go.Scatter(x=ratios["P/E"].index, y=ratios["P/E"].values, name=f"P/E Ratio {ticker}", marker = {'color' : 'springgreen'}), row=4, col=1)
if ticker_2:
    fig.add_trace(go.Scatter(x=ratios_2["P/E"].index, y=ratios_2["P/E"].values, name=f"P/E Ratio {ticker_2}", marker = {'color' : 'mediumseagreen'}), row=4, col=1)

fig.add_trace(go.Scatter(x=ebitda.index, y=ebitda.values, name=f"EBITDA {ticker}", marker = {'color' : 'peru'}), row=4, col=2)
if ticker_2:
    fig.add_trace(go.Scatter(x=ebitda2.index, y=ebitda2.values, name=f"EBITDA {ticker_2}", marker = {'color' : 'sienna'}), row=4, col=2)
    market_cap2 = stock_2.info["marketCap"]

market_cap = stock.info["marketCap"]

fig.update_layout(title=dict(text=f"Financial Dashboard for {ticker}        Market Cap: {market_cap}"),
                  title_subtitle=dict(text=f"(Hover to Inspect, Scroll to Zoom, Double Click to Reset, Click Legend Label to Hide or Display Plot)"),
                  height=800, width=1600, template='plotly_dark')

if ticker_2:
    fig.update_layout(title=dict(text=f"Financial Comparison for {ticker} Market Cap: {market_cap} & {ticker_2} Market Cap: {market_cap2}"),
                      title_subtitle=dict(text=f"(Hover to Inspect, Scroll to Zoom, Double Click to Reset, Click Legend Label to Hide or Display Plot)"),
                  height=800, width=1600, template='plotly_dark')

config = {'scrollZoom': True}
fig.show(config=config)

Enter 'quarters' for past 5 quarters or 'years' for past 5 years (default quarters): 
Enter ticker symbol (default AAPL): 
Optional second ticker for comparison: tsla


#Business Valuation Dashboard

In [ ]:
!pip install dash

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 18.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 228.0/228.0 kB 4.3 MB/s eta 0:00:00
  Attempting uninstall: Werkzeug
    Found existing installation: Werkzeug 3.1.3
    Uninstalling Werkzeug-3.1.3:
      Successfully uninstalled Werkzeug-3.1.3


In [ ]:
import yfinance as yf
import plotly.graph_objects as go
from dash import Dash, dcc, html, Input, Output
import pandas as pd

def calculate_values(ticker, discount_rate, perpetual_growth_rate, revenue_multiple, pe_ratio):
    stock = yf.Ticker(ticker)
    financials = stock.financials
    balance_sheet = stock.balance_sheet
    cashflow = stock.cashflow
    info = stock.info

    values = {}
    equations = {}

    try:
        # 1. Book Value
        total_assets = balance_sheet.loc['Total Assets'][0]
        total_liabilities = balance_sheet.loc['Total Liabilities Net Minority Interest'][0]
        values['Book Value'] = total_assets - total_liabilities
        equations['Book Value'] = "Total Assets - Total Liabilities"

        # 2. Discounted Cash Flow (DCF) Simplified
        free_cash_flow = cashflow.loc['Free Cash Flow'][0]
        values['DCF'] = free_cash_flow * discount_rate
        equations['DCF'] = "Free Cash Flow × Discount Rate"

        # 3. Market Cap
        values['Market Cap'] = info['marketCap']
        equations['Market Cap'] = "Shares Outstanding × Share Price"

        # 4. Enterprise Value (EV)
        ev = info['enterpriseValue']
        values['Enterprise Value'] = ev
        equations['Enterprise Value'] = "Market Cap + Debt - Cash"

        # 5. EBITDA
        values['EBITDA'] = financials.loc["EBITDA"][0]
        equations['EBITDA'] = "Net Income + Interest + Taxes + Depreciation + Amortization"

        # 6. Present Value of Growing Perpetuity
        values['PV of Growing Perpetuity'] = free_cash_flow / (discount_rate - perpetual_growth_rate)
        equations['PV of Growing Perpetuity'] = "Free Cash Flow / (Discount Rate - Growth Rate)"

        # 7. Times Revenue
        revenue = financials.loc['Total Revenue'][0]
        values['Times Revenue'] = revenue * revenue_multiple
        equations['Times Revenue'] = "Total Revenue × Revenue Multiple"

        # 8. Earnings Multiplier
        earnings = financials.loc['Net Income'][0]
        values['Earnings Multiplier'] = earnings * pe_ratio
        equations['Earnings Multiplier'] = "Net Income × P/E Ratio"

        # 9. Liquidity Value
        cash = balance_sheet.loc['Cash And Cash Equivalents'][0]
        values['Liquidity Value'] = cash
        equations['Liquidity Value'] = "Cash and Cash Equivalents"

    except Exception as e:
        print(f"Error calculating value for {ticker}: {e}")

    return values, equations

app = Dash(__name__)

app.layout = html.Div([
    html.H1("Business Valuation Dashboard", style={
        'text-align': 'center',
        'margin-bottom': '10px',
        'color': 'white',
        'font-family': 'Open Sans, sans-serif'
    }),
    html.Div([
        html.Label("Ticker:", style={
            'margin-right': '10px',
            'color': 'white',
            'font-family': 'Open Sans, sans-serif'
        }),
        dcc.Input(
            id="ticker",
            type="text",
            placeholder="AAPL",
            value="AAPL",
            style={
                'width': '80px',
                'background-color': '#2a2a2a',
                'color': 'white',
                'border': '1px solid #444',
                'font-family': 'Open Sans, sans-serif'
            }
        )
    ], style={
        'display': 'flex',
        'align-items': 'center',
        'justify-content': 'center',
        'margin-bottom': '10px'
    }),
    html.Div([
        html.Div([
            html.Label("Discount Rate:", style={
                'font-size': '12px',
                'margin-bottom': '2px',
                'color': 'white',
                'font-family': 'Open Sans, sans-serif'
            }),
            dcc.Slider(
                id="discount_rate",
                min=0.01,
                max=0.2,
                step=0.01,
                value=0.1,
                marks={i / 10: {'label': f"{i*10}%", 'style': {'color': 'white', 'font-family': 'Open Sans, sans-serif'}} for i in range(1, 21)},
                tooltip={"placement": "bottom", "always_visible": False},
                included=False
            )
        ], style={
            'margin-right': '10px',
            'width': '200px'
        }),
        html.Div([
            html.Label("Growth Rate:", style={
                'font-size': '12px',
                'margin-bottom': '2px',
                'color': 'white',
                'font-family': 'Open Sans, sans-serif'
            }),
            dcc.Slider(
                id="growth_rate",
                min=0.01,
                max=0.1,
                step=0.01,
                value=0.03,
                marks={i / 10: {'label': f"{i*10}%", 'style': {'color': 'white', 'font-family': 'Open Sans, sans-serif'}} for i in range(1, 11)},
                tooltip={"placement": "bottom", "always_visible": False}
            )
        ], style={
            'margin-right': '10px',
            'width': '200px'
        }),
        html.Div([
            html.Label("Revenue Multiple:", style={
                'font-size': '12px',
                'margin-bottom': '2px',
                'color': 'white',
                'font-family': 'Open Sans, sans-serif'
            }),
            dcc.Slider(
                id="revenue_multiple",
                min=1,
                max=10,
                step=0.5,
                value=3,
                marks={i: {'label': str(i), 'style': {'color': 'white', 'font-family': 'Open Sans, sans-serif'}} for i in range(1, 11)},
                tooltip={"placement": "bottom", "always_visible": False}
            )
        ], style={
            'margin-right': '10px',
            'width': '200px'
        }),
        html.Div([
            html.Label("P/E Ratio:", style={
                'font-size': '12px',
                'margin-bottom': '2px',
                'color': 'white',
                'font-family': 'Open Sans, sans-serif'
            }),
            dcc.Slider(
                id="pe_ratio",
                min=5,
                max=30,
                step=1,
                value=15,
                marks={i: {'label': str(i), 'style': {'color': 'white', 'font-family': 'Open Sans, sans-serif'}} for i in range(5, 31, 5)},
                tooltip={"placement": "bottom", "always_visible": False}
            )
        ], style={
            'width': '200px'
        })
    ], style={
        'display': 'flex',
        'align-items': 'center',
        'justify-content': 'center',
        'flex-wrap': 'wrap',
        'margin-bottom': '20px'
    }),
    dcc.Graph(
        id="valuation_graph",
        style={'height': '600px'}
    )
], style={
    'background-color': '#000000',
    'padding': '20px',
    'color': 'white',
    'font-family': 'Open Sans, sans-serif'
})

@app.callback(
    Output("valuation_graph", "figure"),
    Input("ticker", "value"),
    Input("discount_rate", "value"),
    Input("growth_rate", "value"),
    Input("revenue_multiple", "value"),
    Input("pe_ratio", "value")
)
def update_dashboard(ticker, discount_rate, growth_rate, revenue_multiple, pe_ratio):
    values, equations = calculate_values(ticker, discount_rate, growth_rate, revenue_multiple, pe_ratio)

    fig = go.Figure()
    grid_positions = [
        {'x': [0, 0.33], 'y': [0.66, 1]},
        {'x': [0.33, 0.66], 'y': [0.66, 1]},
        {'x': [0.66, 1], 'y': [0.66, 1]},
        {'x': [0, 0.33], 'y': [0.33, 0.66]},
        {'x': [0.33, 0.66], 'y': [0.33, 0.66]},
        {'x': [0.66, 1], 'y': [0.33, 0.66]},
        {'x': [0, 0.33], 'y': [0, 0.33]},
        {'x': [0.33, 0.66], 'y': [0, 0.33]},
        {'x': [0.66, 1], 'y': [0, 0.33]}
    ]
    colors = ['lightslategrey', 'steelblue', 'cornflowerblue', 'lightsteelblue', 'paleturquoise', 'lightsteelblue',
              'cornflowerblue', 'steelblue', 'lightslategrey']

    for i, (method, result) in enumerate(values.items()):
        if i < len(grid_positions):
            pos = grid_positions[i]
            equation = equations[method]
            fig.add_trace(go.Indicator(
                mode="number",
                value=result,
                title={"text": f"<span style='color:{colors[i]};'>{method}<br><span style='font-size:12px'>{equation}</span></span>"},
                domain={'x': pos['x'], 'y': pos['y']},
                number={"font": {"color": colors[i]}}
            ))

    fig.update_layout(
        template="plotly_dark",
        height=600
    )
    return fig

if __name__ == "__main__":
    app.run_server(debug=True)


<IPython.core.display.Javascript object>

#Additional Reading for the Curious
Financial Key Performance Indicators <br>
https://online.hbs.edu/blog/post/financial-performance-measures <br>
https://www.investopedia.com/articles/fundamental-analysis/09/five-must-have-metrics-value-investors.asp <br>
Valuing a Business <br>
https://online.hbs.edu/blog/post/how-to-value-a-company <br>
https://www.investopedia.com/terms/b/business-valuation.asp <br>